In [ ]:
import json

class Config:
    """Класс для загрузки и управления конфигурационными данными из JSON файла."""

    def __init__(self, filename):
        """Инициализация объекта Config.
        
        Аргументы:
            filename (str): путь к файлу конфигурации (.json)
        """
        # сохраняем имя файла отдельно
        self.__dict__["_filename"] = filename  
        with open(filename, "r", encoding="utf-8") as f:
            # Загружаем данные из JSON
            self.__dict__["_data"] = json.load(f)

    def __getattr__(self, name):
        """Позволяет доступ к данным через точку (cfg.key)."""
        value = self._data.get(name)
        # Если значение — словарь, создаём вложенный объект Config
        if isinstance(value, dict):
            nested = Config.__new__(Config)  # создаём объект без вызова init
            nested.__dict__["_data"] = value
            nested.__dict__["_filename"] = self._filename
            return nested
        return value

    def __setattr__(self, name, value):
        """Позволяет изменять значения через точку и сохранять в файл."""
        self._data[name] = value
        self._save()

    def _save(self):
        """Сохраняет текущие данные обратно в JSON файл."""
        with open(self._filename, "w", encoding="utf-8") as f:
            json.dump(self._data, f, ensure_ascii=False, indent=4)

    def __repr__(self):
        """Возвращает строковое представление объекта Config."""
        return f"Config({json.dumps(self._data, ensure_ascii=False, indent=2)})"

In [ ]:
### Пример использования


# Создадим тестовый JSON-файл с конфигурацией
config_data = {
    "db": {
        "host": "localhost",
        "port": 5432
    },
    "debug": True
}

with open("config.json", "w", encoding="utf-8") as f:
    json.dump(config_data, f, ensure_ascii=False, indent=4)

# Загружаем конфиг
cfg = Config("config.json")

# Однострочные комментарии:
# Получаем доступ к параметрам через точку
print(cfg.db.host)  # localhost

# Многострочный комментарий:
"""
Изменим значение и убедимся,
что оно сохраняется в файл автоматически.
"""
cfg.debug = False

# Проверим, что данные изменились
print(cfg)